In [1]:
import pandas as pd
import os
import sys

# --- 0. 경로 설정 ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "processed")

# [입력] 원본 수주 공시 목록
ORDERS_FILE = os.path.join(DATA_DIR, "단일판매_공급계약체결_통합.csv")
# [출력] 정제된 "수주 이벤트" 파일
OUTPUT_FILE = os.path.join(DATA_DIR, "orders_processed_plan_b.csv")

print(f"Loading orders file: {ORDERS_FILE}")

# --- 1. Ticker 맵핑 딕셔너리 정의 (기업명 -> 마스터 티커) ---
# (Phase 1-A에서 완성한 맵핑 딕셔너리를 재사용)
NAME_TO_TICKER_MAP = {
    'HD한국조선해양': '042660',
    'HD현대중공업': '009540',
    '삼성중공업': '010140',
    '한화오션': '443060', # '기업명' 컬럼에 '한화오션'이 있는지 확인 필요
    'HD현대미포': '010620', 
    'HD현대마린솔루션': '329180'
    
}

# --- 2. 데이터 로드 및 정제 ---
try:
    df = pd.read_csv(ORDERS_FILE)
    
    print(f"  > Loaded original data: {len(df)} rows")

    # --- 3. [핵심 필터링] '진행사항 공시' 등 노이즈 제거 ---
    # [KEEP] '단일판매ㆍ공급계약체결'이 포함된 공시만 유지
    df_filtered = df[df['공시명'].str.contains('단일판매ㆍ공급계약체결', na=False)].copy()
    
    # [IGNORE] '기타경영사항' 또는 '진행사항 공시'가 포함된 행은 제거 (신규 수주 아님)
    df_filtered = df_filtered[~df_filtered['공시명'].str.contains('기타경영사항|진행사항', na=False)]
    
    print(f"  > Filtered for 'New Contracts' only: {len(df_filtered)} rows")

    # --- 4. Ticker 맵핑 (기업명 -> 마스터 티커) ---
    df_filtered['ticker'] = df_filtered['기업명'].map(NAME_TO_TICKER_MAP)
    
    # [검증] 맵핑 실패한 기업이 있는지 확인
    failed_map = df_filtered[df_filtered['ticker'].isna()]['기업명'].unique()
    if len(failed_map) > 0:
        print(f"\n[!!!] Warning: Ticker 맵핑 실패! (NAME_TO_TICKER_MAP 확인 필요)")
        print(f"  > 맵핑 실패 기업: {failed_map}")
    
    # --- 5. 날짜 변환 (YYYYMMDD -> YYYY-MM-DD) ---
    df_filtered['date'] = pd.to_datetime(df_filtered['공시일자'], format='%Y%m%d')
    
    # --- 6. 'impulse' 채널 생성 (횟수=1) ---
    df_filtered['new_order_event_impulse'] = 1
    
    # --- 7. 최종 컬럼 선택 및 중복 처리 ---
    final_df = df_filtered[['date', 'ticker', 'new_order_event_impulse']]
    
    # [중복 처리] 같은 날, 같은 기업이 여러 건 공시(예: 원본 + 기재정정) 시 
    # 하루에 1번의 이벤트만 카운트
    final_df = final_df.groupby(['date', 'ticker'])['new_order_event_impulse'].max().reset_index()
    
    # --- 8. 저장 ---
    final_df.to_csv(OUTPUT_FILE, index=False)
    
    print(f"\n[SUCCESS] Plan B 수주 데이터 정제 완료!")
    print(f"  > Saved {len(final_df)} events to: {OUTPUT_FILE}")
    print("\n--- Final Orders Data (Head) ---")
    print(final_df.head())
    print("\n--- Final Orders Data (Tail) ---")
    print(final_df.tail())

except FileNotFoundError:
    print(f"\n[!!!] Error: {ORDERS_FILE}을 찾을 수 없습니다.")
except Exception as e:
    print(f"\n[!!!] Error: {e}")

Loading orders file: /workspace/ship-ai/data/processed/단일판매_공급계약체결_통합.csv
  > Loaded original data: 710 rows
  > Filtered for 'New Contracts' only: 671 rows

[SUCCESS] Plan B 수주 데이터 정제 완료!
  > Saved 578 events to: /workspace/ship-ai/data/processed/orders_processed_plan_b.csv

--- Final Orders Data (Head) ---
        date  ticker  new_order_event_impulse
0 2015-01-22  010140                        1
1 2015-03-02  010140                        1
2 2015-03-27  010620                        1
3 2015-04-01  010140                        1
4 2015-04-28  010140                        1

--- Final Orders Data (Tail) ---
          date  ticker  new_order_event_impulse
573 2025-09-30  010140                        1
574 2025-10-20  009540                        1
575 2025-10-20  042660                        1
576 2025-10-22  042660                        1
577 2025-10-24  010140                        1
